## First steps
These are my first steps learning fenics. Since I always learn by example, I'm going to create a single phase flow model in a 1D porous medium. First, I solve the continuity equation, 
$$\nabla.(-\frac{k}{\mu}\nabla p)=0,$$ 
to obtain the pressure profile. Then, I calculate the velocity vector 
$$\mathbf{u_w} = -\frac{k}{\mu}\nabla p$$ 
and use it in the following advection-diffusion equation: 
$$\varphi \frac{\partial c}{\partial t}+\nabla.(\mathbf{u_w}c-\varphi D \nabla c)=0$$
Finally, I compare the results with that of my own finite volume code.

## Continuity equation
The initial condition is $$p(t=0)=p_0,$$ where $p_0=100\times10^5$ Pa. 

In [1]:
from fenics import *

## Not yet thrown away

In [2]:
from dolfin import *

# create mesh and define function space
mesh = UnitSquareMesh(64, 64)
n = FacetNormal(mesh)

V = FunctionSpace(mesh, 'RT', 1)
Q = FunctionSpace(mesh, 'DG', 0)
X = V*Q

# define variational problem
u, p = TrialFunctions(X)
v, q = TestFunctions(X)
g = Constant(0)

k = Max(Expression('exp(-pow((x[1]-0.5-0.1*sin(10*x[0]))/0.1, 2))'), Expression('0.01'))
mu = Constant(1)

# define boundary conditions
pD = Expression('1-x[0]')

# a) variational problem
a = inner(mu/k*u, v)*dx - div(v)*p*dx - div(u)*q*dx
L = g*q*dx - dot(v,n)*pD*ds

# compute solution
xh = Function(X)
solve(a == L, xh)
uh, ph = xh.split()

# plot solution and write output files
plt = plot(ph, mode='color')
plt.write_png('step_06_p')
plt = plot(uh, mode='glyphs')
plt.write_png('step_06_u')

# b) plot the divergence of the velocity field
div_uh = project(div(uh), Q)
plt = plot(div_uh, mode='color')
plt.write_png('step_06_div_u')

TypeError: unsupported operand type(s) for *: 'FunctionSpace' and 'FunctionSpace'

In [5]:
# Tutorial 'Numerical Simulation in Porous Media' (MA 5332)
#
#	Author: Lorenz John, john@ma.tum.de
#          Christian Waluga, christian.waluga@ma.tum.de
#
#

from dolfin import *

# Create mesh
mesh = UnitSquareMesh(32,32)

# Define function spaces and trial/test functions
V = FunctionSpace(mesh, "RT", 1)    # velocity space
Q =FunctionSpace(mesh, "DG", 0)     # pressure space
R = FunctionSpace(mesh, "DG", 0)    # saturation space

VQ = MixedFunctionSpace([V,Q])      # mixed space

(u, p) = TrialFunctions(VQ)
(v, q) = TestFunctions(VQ)

s = TrialFunction(R)
r = TestFunction(R)

# Define time step
c_cfl = 0.025 # max(f') is 20
def get_timestep(U):
   uabs = project(sqrt(dot(U,U)),Q)
   return c_cfl*mesh.hmin()/(max(1.0, max(uabs.vector())))

# Define functions depending on the saturation
# relative permeabilities, linear, quadratic, Brooks-Corey
k_rw = lambda S: S**4
k_rn = lambda S: (1 - S)**2*(1 - S**2)

mu_w = 0.2                                              # viscosity wetting phase
mu_n = 1.0                                              # viscosity non-wetting phase
lambda_ = lambda S: k_rw(S)/mu_w + k_rn(S)/mu_n         # total mobility
F = lambda S: k_rn(S)/(k_rw(S) + mu_w/mu_n * k_rn(S))   # fractional flow function
k = Max(Expression('exp((-1.0) * pow((x[1] - 0.5 - 0.1 * sin(10 * x[0]))/0.1, 2))'),Expression('0.01'))

# Initial condition s(x, t = 0) = 0
s_init = Expression('0')

# Boundary condition for s
class s_boundary(Expression):
    def eval(self, values, x):
        values[0] = 1.0 if near(x[0], 0.0) else 0.0

s_g = s_boundary()

# Boundary condition for p
p_g = Expression('1 - x[0]')

bcs_p = DirichletBC(Q, p_g, 'on_boundary')

# Set parameters
T = 0.2 #0.18

# Mesh-related functions
n = FacetNormal(mesh)       # outer normal vector

# Define functions
U = Function(V)     # velocity, current time step
S = Function(R)     # saturation, current solution
S0 = Function(R)    # saturation, solution of the previous time step
P = Function(Q)     # pressure, current solution

# Set initial condition for the saturation
S0.interpolate(s_init)

# mixed weak form
a = inner(1.0/(k * lambda_(S0)) * v, u) * dx - div(v) * p * dx - q * div(u) * dx
L = -p_g * dot(v,n) * ds

# Time step and update
delta_t = get_timestep(U)
dt = Constant(delta_t)

# Define the weak form for the saturation
a_s = dot(r,s) * dx

un = (dot(U, n) + abs(dot(U, n)))/2
L_0 = dot(r,S0) * dx
L_1 = -dt * dot(grad(r), U*F(S0)) * dx
L_g = avg(dt) * dot(jump(r), un('+')*F(S0('+')) - un('-')*F(S0('-'))) * dS + dt * dot(r, un*F(S0)) * ds
L_s = L_0 + L_1 + L_g

# create output files
#sfile_pvd = File("output/saturation.pvd")
#pfile_pvd = File("output/pressure.pvd")

W = Function(VQ)

# Time-stepping
t = 0
while t < T + DOLFIN_EPS:
#     print 't = %.4f, dt = %.4f' %(t, delta_t)

    # Solve for the velocity and pressure (u, p)
    solve(a == L, W)
    U.assign(W.sub(0, deepcopy=True))
    
    # Calculate the inflow boundary, i.e. u * n < 0
    inflow = lambda x, on_boundary: on_boundary and (\
            (near(x[0],0) and -U(x)[0]  < 0) \
         or (near(x[0],1) and  U(x)[0]  < 0) \
         or (near(x[1],0) and -U(x)[1]  < 0) \
         or (near(x[1],1) and  U(x)[1]  < 0) )
    
    bcs_s = DirichletBC(R, s_g, inflow, 'geometric')
    
    # Solve for the saturation
    solve(a_s == L_s, S, bcs_s)
    
    # Update and move to next time step
    delta_t = get_timestep(U)
    dt.assign(Constant(delta_t))
    t += delta_t
    
    # Limit the saturation to physically meaningful range
    #S.vector()[S.vector() < 0] = 0
    #S.vector()[S.vector() > 1] = 1
    
    plot(S, mode='color')
    
    # Save solution in VTK format in each time step
    #sfile_pvd << S
    #pfile_pvd << P

    S0.assign(S)

interactive()



TypeError: unsupported operand type(s) for *: 'FunctionSpace' and 'FunctionSpace'

In [9]:
# Tutorial 'Numerical Simulation in Porous Media' (MA 5332)
#
# Note: This is only a code template which has to be extended and
#       significantly improved in the tutorial!
#
#	Author: Lorenz John, john@ma.tum.de
#          Christian Waluga, christian.waluga@ma.tum.de
#

from dolfin import *

# Create mesh
mesh = UnitSquareMesh(32,32)

# Define function spaces and trial/test functions
V = VectorFunctionSpace(mesh, "CG", 1)      # velocity space
Q = FunctionSpace(mesh, "CG", 1)            # pressure  space
R = FunctionSpace(mesh, "DG", 0)            # saturation space

p = TrialFunction(Q)
q = TestFunction(Q)

s = TrialFunction(R)
r = TestFunction(R)

# Define time step
def get_timestep(U):
    return 0.001

# Define functions depending on the saturation
# relative permeabilities Brooks-Corey
k_rw = lambda S: S**4
k_rn = lambda S: (1 - S)**2*(1 - S**2)

mu_w = 0.2                                              # viscosity wetting phase
mu_n = 1.0                                              # viscosity non-wetting phase
lambda_ = lambda S: k_rw(S)/mu_w + k_rn(S)/mu_n         # total mobility
F = lambda S: k_rn(S)/(k_rw(S) + mu_w/mu_n * k_rn(S))   # fractional flow function
u = lambda S,P: -k * lambda_(S) * grad(P)               # velocity
# k = Max(Expression('exp((-1.0) * pow((x[1] - 0.5 - 0.1 * sin(10 * x[0]))/0.1, 2))'),Expression('0.01'))
k=Expression('0.01', degree=1)
# Initial condition s(x, t = 0) = 0
s_init = Expression('0', degree=1)

# Boundary condition for s
class s_boundary(Expression):
    def eval(self, values, x):
        if near(x[0], 0.0):
            values[0] = 1.0
        else:
            values[0] = 0.0

s_g = s_boundary()

# Boundary condition for p
p_g = Expression('1 - x[0]')
bcs_p = DirichletBC(Q, p_g, 'on_boundary')

f = Expression('0.0')

# Set parameters
T = 0.15

U = interpolate(Constant((0,0)), V)
delta_t = get_timestep(U)
dt = Constant(delta_t)

# Mesh-related functions
n = FacetNormal(mesh)

# Define functions
S = Function(R)     # saturation, current solution
S0 = Function(R)    # saturation, solution of the previous time step
P = Function(Q)     # pressure, current solution

# Set initial condition for the saturation
S0.interpolate(s_init)

# Define the weak form of the problem in (u,p)
a = inner(grad(q), k * lambda_(S0) * grad(p)) * dx
L = inner(q, f) * dx

# Define the weak form for the saturation
a_s = dot(r,s) * dx

# Time step and update
delta_t = get_timestep(U)
dt.assign(delta_t)

un = (dot(U, n) + abs(dot(U, n)))/2.0
L_0 = dot(r,S0) * dx
L_1 = -dt * dot(grad(r), U*F(S0)) * dx
L_g = avg(dt) * dot(jump(r), un('+')*F(S0('+')) - un('-')*F(S0('-'))) * dS + dt * dot(r, un*F(S0)) * ds
L_s = L_0 + L_1 + L_g

# Time-stepping
t = 0
while t < T + DOLFIN_EPS:
#     print 't = %.4f, dt = %.4f' %(t, delta_t)
    
    # Solve for the pressure p
    solve(a == L, P, bcs_p)

    # Update the velocity
    U.assign(project(u(S0,P), V))

    # Calculate the inflow boundary, i.e. u * n < 0
    inflow = lambda x, on_boundary: on_boundary and (\
            (near(x[0],0) and -U(x)[0]  < 0) \
         or (near(x[0],1) and  U(x)[0]  < 0) \
         or (near(x[1],0) and -U(x)[1]  < 0) \
         or (near(x[1],1) and  U(x)[1]  < 0) )
    
    # set boundary conditions
    bcs_s = DirichletBC(R, s_g, inflow, 'geometric')

    # Solve for the saturation
    solve(a_s == L_s, S, bcs_s)
    
    # Update and move to next time step
    delta_t = get_timestep(U)
    dt.assign(delta_t)
    t += delta_t

    S0.assign(S)


RuntimeError: Must supply C++ code to Expression. You may want to use UserExpression